In [ ]:
# anxiety-depression-support
# mental-health-support
# anxietysupport
# mdd-support

In [1]:
import requests
import json
import os
import pandas as pd
import re
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
def write_list_to_file(file_name, write_list):
    
    with open(file_name, 'w') as fp:
        fp.write('\n'.join(write_list))   
  

In [17]:
def read_list_from_file(file_name):

    if os.path.exists(file_name):
        with open(file_name, 'r') as fp:
            read_list = fp.read().split("\n")
            
            return read_list
    else:
        return []

In [5]:
def add_posts_to_file(file_name, posts):
    
    file_posts = read_list_from_file(file_name)
    file_posts.extend(posts)
    file_posts = list(set(file_posts))
    write_list_to_file(file_name, file_posts)
    

In [6]:
def get_page(session_cookie, page_number):
    
    url = "https://healthunlocked.com/private/posts/anxiety-depression-support/latest?pageNumber={pageNumber}".format(pageNumber = page_number)

    payload={}
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
      'Accept-Language': 'it-IT,it;q=0.8,en-US;q=0.5,en;q=0.3',
      'Accept-Encoding': 'gzip, deflate, br',
      'Connection': 'keep-alive',
      'Cookie': 'huBv=B; huSessID={huSessID};'.format(huSessID = session_cookie),
      'Upgrade-Insecure-Requests': '1',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response


In [7]:
def get_post(session_cookie, post_id):
    
    url = "https://healthunlocked.com/private/posts/anxiety-depression-support/{id}".format(id = post_id)

    payload={}
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
      'Accept-Language': 'it-IT,it;q=0.8,en-US;q=0.5,en;q=0.3',
      'Accept-Encoding': 'gzip, deflate, br',
      'Connection': 'keep-alive',
      'Cookie': 'huBv=B; huSessID={huSessID};'.format(huSessID = session_cookie),
      'Upgrade-Insecure-Requests': '1',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response

   

In [8]:
def save_post(session_cookie, post_id, file_name):
    
    response = get_post(session_cookie, post_id)
    
    if response.status_code == 200:
        post = json.loads(response.text)
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(post, f, ensure_ascii=False)
    else:
        print(response.status_code, response.text)       
        

In [9]:
def clean_text(text):

    # Remove line breaks
    text = re.sub(r'[^a-zA-Z0-9 ,\.\?!]', '', text)

    return text


In [10]:
def save_latest_posts(session_cookie, number_of_pages, posts_text_file):

    # range must start from 1
    for page_number in range(1, number_of_pages):
        
        time.sleep(10)
        
        response = get_page(session_cookie, page_number)
        
        if response.status_code == 200:
            page = json.loads(response.text)
            posts = [str(p['postId']) for p in page]
            add_posts_to_file(posts_text_file, posts)
            if page_number % 10 == 0:
                print('Saved posts for page {}'.format(page_number))
        else:
            print(response.status_code, response.text)
    
    

In [11]:
def save_posts_as_json(session_cookie, posts_ids):
    
    for post_id in posts_ids:
        file_name = "health-data/posts/{id}.json".format(id = post_id)
        if os.path.exists(file_name) == False:
            time.sleep(10)
            save_post(session_cookie, post_id, file_name)
                

In [12]:
def save_posts_to_csv(file_name, post_ids, append):
    
    posts_df = pd.DataFrame(columns = ['id','dateCreated','dateLastActivity','author.id','author.age','author.gender', 'author.country','postUrl','body'],)
    
    if append == True and os.path.exists(file_name):
        df = pd.read_csv(file_name)
        posts_df = pd.concat([posts_df, df]) 

    for post_id in post_ids:
        post_file_name = "health-data/posts/{id}.json".format(id = post_id)
        if os.path.exists(post_file_name) == True and post_id not in posts_df['id'].values:

            # Opening JSON file
            with open(post_file_name,"r",encoding='utf8') as f:
                # returns JSON object as a dictionary
                post = json.load(f)
                #post['body'] = clean_text(post['body'])
                df = pd.DataFrame(pd.json_normalize(post))
                # Filter columns
                df = df[list(posts_df)]
                #df['body'] = df['body'].apply(lambda x: clean_text(x))
                posts_df = pd.concat([posts_df, df])
                    

    posts_df.to_csv(file_name, index=False)
    

In [13]:
# initialize the Chrome driver    
driver = webdriver.Chrome()

# login to get session cookie
driver.get("https://healthunlocked.com/login/user/private")   
   

In [14]:
session_cookie = driver.get_cookie("huSessID")["value"]

In [15]:
print(session_cookie)

9d4d4788-8252-471e-856e-83a954feacae%3A3b39f1ee-b9db-4145-9e5e-ae49a2b31e5a


In [22]:
# Save list of latest posts to text file

#session_cookie = get_session_cookie()
page_number = 2

# Add latest posts to list
save_latest_posts(session_cookie, page_number, 'health-data/posts.txt')

In [23]:
# Save json of new posts

# Get list of posts
posts = read_list_from_file('health-data/posts.txt')
posts_ids = [f for f in posts]

# Save as json
save_posts_as_json(session_cookie, posts_ids)

In [21]:
# Add posts to csv file
posts = read_list_from_file('health-data/posts.txt')
post_ids = [f for f in posts]
file_name = "health-data/posts.csv"
save_posts_to_csv(file_name = file_name, post_ids = post_ids , append = os.path.exists(file_name))

In [ ]:
# Check if there posts to download

# all posts
posts = read_list_from_file('health-data/posts.txt')
posts_list = [int(f) for f in posts]
print("All posts {}".format(len(posts)))

# list of json files
json_file_list = os.listdir('health-data/posts')
json_list = [int(f.replace('.json','')) for f in json_file_list]
print("Json files: {}".format(len(json_list)))
      
#  list processed posts
posts_df = pd.read_csv('health-data/posts.csv')
ids_list = posts_df['id'].values.tolist()
print("Processed posts: {}".format(len(ids_list)))
      
# dictionary to search for already processed posts
ids = posts_df['id'].values 
ids_dict = {ids[i]: i for i in range(len(ids))}

# posts to download
posts_to_download = list(set(posts_list) - set(json_list))
print("Posts to download: {}".format(len(posts_to_download)))

# json files not processed
posts_to_process = list(set(json_list) - set(ids_list))
print("Posts to process: {}".format(len(posts_to_process)))

#posts_to_process_list = [str(p) for p in posts_to_process]
#write_list_to_file('posts_to_process.txt',posts_to_process_list)

In [ ]:
posts_df = pd.read_csv('posts.csv')

In [ ]:
posts_df.head()